In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('prayers_data.xlsx')

In [3]:
df.columns

Index(['S.NO', 'CASE NO', 'PETITIONER ADV.', 'RESPONDENT ADV.', 'PETITIONER',
       'RESPONDENT', 'SUBJECT CATEGORY', 'PRAYER'],
      dtype='object')

In [4]:
from langchain_community.llms import Ollama
model_name = "phi3"
llm = Ollama(model=model_name)

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="phi3")

In [6]:
len(embeddings.embed_query(df['PRAYER'][0]))

3072

In [1]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

In [2]:
# Connect to Milvus
connections.connect(
    host="localhost", # Replace with your Milvus server IP
    port="19530"
)

In [6]:
df.columns

NameError: name 'df' is not defined

In [7]:
# Create schema
prayers_fields = [
    FieldSchema(name="prayerId", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="prayerVector", dtype=DataType.FLOAT_VECTOR, dim=3072), # Vector field for film vectors
    FieldSchema(name="prayerText", dtype=DataType.VARCHAR, max_length=20000),
    FieldSchema(name="CASE_NO", dtype=DataType.VARCHAR, max_length=400),
    FieldSchema(name="PETITIONER_ADV", dtype=DataType.VARCHAR, max_length=400),
    FieldSchema(name="RESPONDENT_ADV", dtype=DataType.VARCHAR, max_length=400),
    FieldSchema(name="PETITIONER", dtype=DataType.VARCHAR, max_length=400),
    FieldSchema(name="RESPONDENT", dtype=DataType.VARCHAR, max_length=400),
    FieldSchema(name="SUBJECT_CATEGORY", dtype=DataType.VARCHAR, max_length=400)]

In [8]:
prayers_schema = CollectionSchema(fields=prayers_fields,enable_dynamic_field=False)

In [15]:
# Create collection
collection = Collection(name="prayers_collection_updated", schema=prayers_schema)

RPC error: [drop_collection], <ParamError: (code=1, message=`timeout` value prayers_collection_2 is illegal)>, <Time:{'RPC start': '2024-08-29 03:32:04.961101', 'RPC error': '2024-08-29 03:32:04.961143'}>


ParamError: <ParamError: (code=1, message=`timeout` value prayers_collection_2 is illegal)>

In [3]:
from pymilvus import utility

# List all collections
collections = utility.list_collections()
print("Available Collections:")
for collection in collections:
    print(collection)


Available Collections:
vector_collection
order_collection
section_collection
JudgementCollection


In [4]:
from pymilvus import Collection

# Specify the collection name
collection_name = "JudgementCollection"  # Replace with your collection name

# Load the collection
collection = Collection(name=collection_name)

# Print the schema
print(f"Schema of collection '{collection_name}':")
print(collection.schema)

Schema of collection 'JudgementCollection':
{'auto_id': True, 'description': 'Collection for storing judgement data', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'judgementChunkId', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'judgementVector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3072}}, {'name': 'judgementText', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 20000}}, {'name': 'judgementFileName', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 255}}], 'enable_dynamic_field': False}


In [5]:
# Count the number of entities in the collection
entity_count = collection.num_entities
print(f"Total number of entities in collection '{collection_name}': {entity_count}")

Total number of entities in collection 'JudgementCollection': 235


In [13]:
# Create index for each vector field
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128},
}

In [14]:
collection.create_index("prayerVector", index_params)

Status(code=0, message=)

In [15]:
from joblib import Parallel, delayed

In [20]:
# Define the function that will process each row
def process_row(idx, row):
    return {"prayerId": idx,
            "prayerVector":embeddings.embed_query(str(row['PRAYER'])), 
            "prayerText": str(row['PRAYER']),
            "CASE_NO":row['CASE NO'],
            "PETITIONER_ADV":row['PETITIONER ADV.'],
            "RESPONDENT_ADV":row['RESPONDENT ADV.'],
            "PETITIONER":row['PETITIONER'],
            "RESPONDENT":row["RESPONDENT"],
            "SUBJECT_CATEGORY":row['SUBJECT CATEGORY'] 
            }

In [17]:
df.columns

Index(['S.NO', 'CASE NO', 'PETITIONER ADV.', 'RESPONDENT ADV.', 'PETITIONER',
       'RESPONDENT', 'SUBJECT CATEGORY', 'PRAYER'],
      dtype='object')

In [ ]:
from huggingface_hub import notebook_login
from transformers import AutoModelForSequenceClassification, AutoTokenizer

notebook_login()  # Logs into Hugging Face

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Push to Hub
model.push_to_hub("your-username/your-model-name")
tokenizer.push_to_hub("your-username/your-model-name")


In [21]:
# Use joblib's Parallel and delayed to parallelize the loop
results = Parallel(n_jobs=-1)(delayed(process_row)(idx, row) for idx, row in df.iterrows())


KeyboardInterrupt: 

In [1]:
data = list(results)

NameError: name 'results' is not defined

In [ ]:
data[0]

{'prayerId': 0,
 'prayerVector': [-1.7440944910049438,
  0.2282790243625641,
  1.1182295083999634,
  2.293494462966919,
  1.0966609716415405,
  1.0159319639205933,
  -0.8897358775138855,
  1.2843018770217896,
  1.1066224575042725,
  1.3042434453964233,
  0.21765252947807312,
  0.21947723627090454,
  -1.0339727401733398,
  0.8158777356147766,
  1.8304885625839233,
  -2.219592571258545,
  0.6633901596069336,
  2.533487319946289,
  -1.5494542121887207,
  1.3302439451217651,
  -0.9630224108695984,
  -0.36090603470802307,
  1.0059053897857666,
  3.4548232555389404,
  1.8508564233779907,
  0.6503227949142456,
  -1.1117910146713257,
  0.2225014865398407,
  -0.5437605977058411,
  -0.6663047671318054,
  1.2761003971099854,
  -0.691794753074646,
  0.07342290878295898,
  -0.8762263655662537,
  1.148814082145691,
  -0.9223611950874329,
  -0.21998541057109833,
  -1.242414116859436,
  1.952113151550293,
  -2.9527087211608887,
  0.0899672657251358,
  -2.820596694946289,
  -0.11082655936479568,
  0.13

In [ ]:
chunk_size = 10
for i in range(0, len(data), chunk_size):
    collection.insert(data[i:i + chunk_size])

In [6]:
from pymilvus import Collection

# Load the collection by its name
collection_name = "prayers_collection_updated"  # Replace with your collection name
collection = Collection(name=collection_name)
collection.load()

# Get the total number of entities in the collection
total_records = collection.num_entities

# Print the total number of records
print(f"Total number of records in the collection: {total_records}")

Total number of records in the collection: 4515
